In [1]:
import tensorflow as tf
import keras_tuner as kt
import sys
sys.path.append("..")
import tools
%load_ext autoreload
%autoreload 2

In [2]:
dataset_train = tf.data.TFRecordDataset(["../DATA/train1.tfrecord"])
tfrecord_shape = tools.model.get_shape_of_quadratic_image_tfrecord(dataset_train)
dataset_train = dataset_train.map(tools.model.parse_function(img_shape=tfrecord_shape, test=False))
dataset_train = dataset_train.shuffle(5 * 128).batch(128).prefetch(2)
dataset_val = tf.data.TFRecordDataset(["../DATA/test1.tfrecord"])
dataset_val = dataset_val.map(tools.model.parse_function(img_shape=tfrecord_shape, test=False))
dataset_val = dataset_val.batch(128).prefetch(2)
strategy = tf.distribute.get_strategy()
FE = tf.keras.losses.BinaryFocalCrossentropy(apply_class_balancing=True, alpha=0.95)
CE = tf.keras.losses.BinaryCrossentropy()
earlystopping_kb = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5 * 6,
                                                    verbose=1,
                                                    restore_best_weights=True)
terminateonnan_kb = tf.keras.callbacks.TerminateOnNaN()
reducelronplateau_kb = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.95,
                                                            patience=6, verbose=1)
tuner = kt.Hyperband(hypermodel=tools.hypertuneModels.StockHyperModel(tfrecord_shape, tools.model.custom_loss_sum([FE, CE])),
                    objective=kt.Objective('val_f1_score', "max"),
                    max_epochs=64,
                    factor=int(4),
                    hyperband_iterations=64,
                    directory="/astro/users/kmrakovc/Projects/Tuner",
                    overwrite=False,
                    project_name="AsteroidsNN_Tuner",
                    distribution_strategy=strategy)

In [ ]:
best_hps = tools.hypertuneModels.get_best_hyperparameters(tuner, num_trials=10)

In [ ]:
best_hps